In [ ]:
# SCIPY implimentation segment starting with Gaussians python script from Itu:
import math
import random
from scipy.optimize import curve_fit
from scipy.special import voigt_profile
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
# plt.rcParams['figure.dpi']=300 # highres display
from specutils import Spectrum1D
from specutils.fitting import fit_generic_continuum as fgc
from astropy import units as u
from astropy.visualization import quantity_support
import time
quantity_support()
from lmfit import Model
import pathlib
import pandas as pd

# Script to perform multiple Gaussian/Lorentzian/Voigt fits on the, single/double peak H-alpha line (on the blue, center and red peak)

file = np.loadtxt(r".\\GX304-1\\smbxgpP201903220040_cr_cg_wr_01.txt") # smbxgpP201206270016_cr_cg_wr_01.txt smbxgpP201401200077_cr_cg_wr_01.txt smbxgpP202001120152_cr_cg_wr_01.txt
Source_Name="GX304-1"
x = file[:,0]
y = file[:,1]


def func_gaus(x, ctr, amp, wid): #Fitting function gaussian
    y = np.zeros_like(x)
    y = y + amp * np.exp( -((x - ctr)**2/wid**2))
    return y
def func_DoubleGaus(x, ctr, amp, wid,ctr2,amp2,wid2): #Fitting function gaussian
    y = np.zeros_like(x)
    y = y + amp * np.exp( -((x - ctr)**2/wid**2)) + amp2 * np.exp( -((x - ctr2)**2/wid2**2))
    return y

def func_lorentzian(x, ctr, amp, wid): #Fitting function lorentzian
    y = np.zeros_like(x)

    y = y + amp * (0.5*wid**2/((x-ctr)**2+(0.5*wid)**2))
    return y
def func_DoubleLorentzian(x, ctr, amp, wid, ctr2, amp2, wid2): #Fitting function lorentzian
    y = np.zeros_like(x)

    y = y + amp * (0.5*wid**2/((x-ctr)**2+(0.5*wid)**2)) + amp2 * (0.5*wid2**2/((x-ctr2)**2+(0.5*wid2)**2))
    return y

def func_voigt(x, ctr, amp, wid): #Fitting function voigt

    y = np.zeros_like(x)

    gam = wid-1
    y = y + voigt_profile(x - ctr, wid, gam) * amp
    return y
def func_DoubleVoigt(x, ctr, amp, wid, ctr2, amp2, wid2): #Fitting function voigt

    y = np.zeros_like(x)

    gam = wid-1
    gam2 = wid2-1
    y = y + voigt_profile(x - ctr, wid, gam) * amp + voigt_profile(x - ctr2, wid2, gam2) * amp2
    return y

# def find_double_peak_coords(x_data,y_data): #Function to find P_0 guess values for double peak plotting
    halpha_region_x = x_data[(x_data > 6540) & (x_data < 6590)]
    halpha_region_y = y_data[(x_data > 6540) & (x_data < 6590)]

    # print(halpha_region_x)

    peaks, props = find_peaks(halpha_region_y)
    # print(peaks)
    peak_wavelengths=halpha_region_x[peaks]
    peak_fluxes=halpha_region_y[peaks]
    # print("The peak Wl and fluxes: ",peak_wavelengths,peak_fluxes)
    
    valleys, props = find_peaks(-halpha_region_y)
    # print(valleys)
    valley_wavelength=halpha_region_x[valleys]
    valley_flux=halpha_region_y[valleys]
    # print(valley_wavelength,valley_flux)

    # --- Step 3: Pick the two largest peaks + the valley between
    # Sort peaks by flux
    sorted_peaks = np.argsort(peak_fluxes)[-2:]  # take top 2
    selected_peaks = [(peak_wavelengths[i], peak_fluxes[i]) for i in sorted_peaks]

    # For the dip, just take min flux between the two peaks
    left, right = np.min([p[0] for p in selected_peaks]), np.max([p[0] for p in selected_peaks])
    mask = (halpha_region_x > left) & (halpha_region_x < right)
    dip_idx = np.argmin(halpha_region_y[mask])
    dip_wavelength = halpha_region_x[mask][dip_idx]
    dip_flux = halpha_region_y[mask][dip_idx]

    # --- Print results ---
    print("Peaks (λ, flux):", selected_peaks)
    print("Dip (λ, flux):", (dip_wavelength, dip_flux))

    ctr1=math.trunc(selected_peaks[0][0]*10)
    amp1=math.trunc(selected_peaks[0][1]*10)
    wid1=6*10
    ctr2=valley_wavelength
    ctr2=math.trunc(ctr2[0]*10)
    wid2=0.5*((valley_wavelength-peak_wavelengths[0])+(peak_wavelengths[1]-valley_wavelength))
    wid2=math.trunc(3*10)
    ctr3=math.trunc(selected_peaks[1][0]*10)
    amp3=math.trunc(selected_peaks[1][1]*10)
    wid3=4*10
    amp2=dip_flux
    # amp2=-((amp1/10+amp3/10)/2-amp2)*5
    amp2=math.trunc(amp2*-0.76689539232*10)

    result=[ctr1/10,amp1/10,wid1/10,ctr2/10,amp2/8,wid2/10,ctr3/10,amp3/10,wid3/10]
    # result=[ctr1/10,amp1/10-1,wid1/10,ctr3/10,amp3/10-1,wid3/10]

    # if len(peaks)==1:
    #     # print(peak_wavelengths[0])
    #     ctr1=math.trunc(peak_wavelengths[0]*10)
    #     amp1=math.trunc(peak_fluxes[0]*10)
    #     wid1=13.0*10

    #     result=[ctr1/10,amp1/10-1,wid1/10]
    #     print(result)

    return result

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def Smooth_find_single_valley_coords(x_data,y_data): #Function to find P_0 guess values for single valley plotting
    print("We are in the valley finder!")

    valleys, _ = find_peaks(-y_data,distance=3)
    print(valleys)
    peak_wavelengths=x_data[valleys]
    peak_fluxes=y_data[valleys]
    if len(valleys)>1:
        # Sort peaks by flux
        sorted_peaks = np.argsort(peak_fluxes)[-2:]  # take top 2
        selected_peaks = [(peak_wavelengths[i], peak_fluxes[i]) for i in sorted_peaks]

        # For the valley, just take min flux between the two peaks
        left, right = np.min([p[0] for p in selected_peaks]), np.max([p[0] for p in selected_peaks])
        mask = (x_data >= left) & (x_data <= right)
        dip_idx = np.argmin(y_data[mask])
        dip_wavelength = x_data[mask][dip_idx]
        dip_flux = y_data[mask][dip_idx]

        print("Peaks (λ, flux):", selected_peaks)
        print("Valley (λ, flux):", (dip_wavelength, dip_flux))

        ctr1=math.trunc(selected_peaks[0][0]*10)
        amp1=math.trunc(selected_peaks[0][1]*10)
        wid1=6*10
        ctr2=dip_wavelength
        ctr2=math.trunc(ctr2*10)
        wid2=0.3*((dip_wavelength-peak_wavelengths[0])+(peak_wavelengths[1]-dip_wavelength))
        wid2=math.trunc(3*10)
        ctr3=math.trunc(selected_peaks[1][0]*10)
        amp3=math.trunc(selected_peaks[1][1]*10)
        wid3=4*10
        amp2=dip_flux
        # amp2=-((amp1/10+amp3/10)/2-amp2)*5
        amp2=math.trunc(amp2*0.76689539232*10)

        result=[ctr1/10,amp1/10,wid1/10,ctr2/10,amp2/10,wid2/10,ctr3/10,amp3/10,wid3/10]
        # result=[ctr1/10,amp1/10-1,wid1/10,ctr3/10,amp3/10-1,wid3/10]

    if len(valleys)==1:
        # print(peak_wavelengths[0])
        ctr1=math.trunc(peak_wavelengths[0]*10)
        amp1=math.trunc(peak_fluxes[0]*10)
        wid1=5.0*10

        result=[ctr1/10,amp1/10,wid1/10]
    # ctr1=math.trunc(peak_wavelengths[0]*10)
    # amp1=math.trunc(peak_fluxes[0]*10)
    # wid1=5.0*10

    # result=[ctr1/10,amp1/10,wid1/10]
    # print(result)

    return result

def Smooth_find_double_peak_coords(x_data,y_data): #Function to find P_0 guess values for double or single peak plotting
    halpha_region_x = x_data[(x_data > 6553) & (x_data < 6572)]
    halpha_region_y = y_data[(x_data > 6553) & (x_data < 6572)]

    halpha_region_y=smooth(halpha_region_y,6)

    peaks, _ = find_peaks(halpha_region_y,distance=3)
    print(peaks)
    peak_wavelengths=halpha_region_x[peaks]
    peak_fluxes=halpha_region_y[peaks]

    if len(peaks)>1:
        # Sort peaks by flux
        sorted_peaks = np.argsort(peak_fluxes)[-2:]  # take top 2
        selected_peaks = [(peak_wavelengths[i], peak_fluxes[i]) for i in sorted_peaks]

        # For the valley, just take min flux between the two peaks
        left, right = np.min([p[0] for p in selected_peaks]), np.max([p[0] for p in selected_peaks])
        mask = (halpha_region_x >= left) & (halpha_region_x <= right)
        dip_idx = np.argmin(halpha_region_y[mask])
        dip_wavelength = halpha_region_x[mask][dip_idx]
        dip_flux = halpha_region_y[mask][dip_idx]

        # print("Peaks (λ, flux):", selected_peaks)
        # print("Valley (λ, flux):", (dip_wavelength, dip_flux))

        ctr1=math.trunc(selected_peaks[0][0]*10)
        amp1=math.trunc(selected_peaks[0][1]*10)
        wid1=6*10
        ctr2=dip_wavelength
        ctr2=math.trunc(ctr2*10)
        wid2=0.3*((dip_wavelength-peak_wavelengths[0])+(peak_wavelengths[1]-dip_wavelength))
        wid2=math.trunc(3*10)
        ctr3=math.trunc(selected_peaks[1][0]*10)
        amp3=math.trunc(selected_peaks[1][1]*10)
        wid3=4*10
        amp2=dip_flux
        # amp2=-((amp1/10+amp3/10)/2-amp2)*5
        amp2=math.trunc(amp2*0.76689539232*10)

        # result=[ctr1/10,amp1/10,wid1/10,ctr2/10,amp2/10,wid2/10,ctr3/10,amp3/10,wid3/10]
        result=[ctr1/10,amp1/10-1,wid1/10,ctr3/10,amp3/10-1,wid3/10]
        print("3 component guess: ",result)

    if len(peaks)==1:
        # print(peak_wavelengths[0])
        ctr1=math.trunc(peak_wavelengths[0]*10)
        amp1=math.trunc(peak_fluxes[0]*10)
        wid1=5.0*10

        result=[ctr1/10,amp1/10,wid1/10]
        print("Single peak guess: ",result)

    if len(peaks)==0:
        result=Smooth_find_single_valley_coords(halpha_region_x,halpha_region_y)
        print("result found no peaks, maybe only one big valley?")

    return result

def spec_normalisation(x,y):


    ########### Function to normalise spectrum by dividing with fitted continuum (NTE this should only be done if imported data is not already normalised):
    s1_cal=y*u.Unit('erg cm-2 s-2 AA-1') #flux data
    wav_cal = x*u.AA #wavelength data

    spec=Spectrum1D(spectral_axis=wav_cal,flux=s1_cal)
    s_fit=fgc(spec,median_window=1)
    y_cont_fitted=s_fit(wav_cal)
    print(len(spec.spectral_axis.value))
    # print(find_double_peak_coords(spec.spectral_axis.value,(spec.flux/y_cont_fitted).value))


    fig=plt.figure(figsize=(8,5)) #create the figure
    # plt.yscale("log") #set y scale to log to correctly display the spectra
    plt.plot(spec.spectral_axis, spec.flux, label='spectra')
    plt.plot(wav_cal, y_cont_fitted, label='fitted continuum')
    plt.legend(fontsize=Legend_font_size)
    plt.show()
    plt.close()

    #now plot normalised spectra
    fig=plt.figure(figsize=(8,5)) #create the figure

    plt.plot(spec.spectral_axis, spec.flux/y_cont_fitted, label='Normalized spectra')
    plt.legend(fontsize=Legend_font_size)
    # plt.yscale("log") #set y scale to log to correctly display the spectra
    plt.show()
    plt.close()
    return spec.spectral_axis.value, (spec.flux/y_cont_fitted).value
    ###########

def fit_ALL_t1(Source_Name, nameString, guess, x, y):
    ########### Gaussian fitting and plot:
    popt, pcov = curve_fit(func_gaus, x, y, p0=guess,maxfev=50000)
    # print(popt)
    fit_GAUS = func_gaus(x, *popt)
    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.grid('minor')
    plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='H{alpha}')
    plt.plot(x, y, 'r-', linewidth=1,label='Normalised Spectra')
    plt.plot(x, fit_GAUS, 'b-', linewidth=3, alpha=0.5,label='Gaussian Fit')
    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_gaus(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()
    ###########

    ########### Lorentzian fitting and plot:
    popt, pcov = curve_fit(func_lorentzian, x, y, p0=guess,maxfev=50000)
    # print(popt)
    fit_LORERTZIAN = func_lorentzian(x, *popt)

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.grid('minor')
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_LORERTZIAN , 'b-', linewidth=3, alpha=0.5, label='Lorentzian Fit')

    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_lorentzian(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()
    ###########

    ########### Average between GAUS and LORERNTZ fits, plot only:
    fit_AVG=(fit_GAUS+fit_LORERTZIAN)/2

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_AVG , 'b-', linewidth=3, alpha=0.5,label='average between Gaussian and Lorentzian Fit')
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_Average_Gaus_Lor_Fit.png')
    plt.close()
    ###########

    ########### Voigt fitting and plot:
    popt, pcov = curve_fit(func_voigt, x, y, p0=guess,maxfev=50000)
    fit_VOIGT = func_voigt(x, *popt)

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.grid('minor')
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_VOIGT , 'b-', linewidth=3, alpha=0.5,label='Voigt Fit')

    
    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_voigt(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()
    ###########

    
    return ("finished "+nameString)

def fit_ALL_t2(Source_Name, nameString, guess, x, y):
    ########### Gaussian fitting and plot:
    popt, pcov = curve_fit(func_DoubleGaus, x, y, p0=guess,maxfev=50000)
    # print(popt)
    fit_GAUS = func_DoubleGaus(x, *popt)
    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.grid('minor')
    plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='H{alpha}')
    plt.plot(x, y, 'r-', linewidth=1,label='Normalised Spectra')
    plt.plot(x, fit_GAUS, 'b-', linewidth=3, alpha=0.5,label='Gaussian Fit')
    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_gaus(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()
    ###########

    ########### Lorentzian fitting and plot:
    popt, pcov = curve_fit(func_DoubleLorentzian, x, y, p0=guess,maxfev=50000)
    # print(popt)
    fit_LORERTZIAN = func_DoubleLorentzian(x, *popt)

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.grid('minor')
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_LORERTZIAN , 'b-', linewidth=3, alpha=0.5, label='Lorentzian Fit')

    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_lorentzian(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()
    ###########

    ########### Average between GAUS and LORERNTZ fits, plot only:
    fit_AVG=(fit_GAUS+fit_LORERTZIAN)/2

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_AVG , 'b-', linewidth=3, alpha=0.5,label='average between Gaussian and Lorentzian Fit')
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_Average_Gaus_Lor_Fit.png')
    plt.close()
    ###########

    ########### Voigt fitting and plot:
    popt, pcov = curve_fit(func_DoubleVoigt, x, y, p0=guess,maxfev=50000)
    fit_VOIGT = func_DoubleVoigt(x, *popt)

    plt.figure(figsize=(graph_W,graph_H)) #create the figure
    plt.grid('minor')
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6553) & (x < 6572)]))+0.2)
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, fit_VOIGT , 'b-', linewidth=3, alpha=0.5,label='Voigt Fit')

    
    c=1
    for i in range(0, len(popt), 3):
        y_temp = np.zeros_like(x)
        ctr = popt[i]
        amp = popt[i+1]
        wid = popt[i+2]
        y_temp = func_voigt(x, ctr, amp, wid)
        plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label=f'individual Component {c}')
        c=c+1
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()
    ###########

    
    return ("finished "+nameString)

xmin,xmax = 6550,6575
ymin,ymax=-4.5,2
graph_W,graph_H = 8,5

fig=plt.figure(figsize=(graph_W,graph_H))
plt.xlim(xmin,xmax)
plt.plot(x, y,'o')

plt.plot(x, smooth(y,3), 'ro', lw=2)

plt.plot(x, smooth(y,8), 'go', lw=1.5)
plt.show()
plt.close()


In [ ]:
########### SCIPY Gaussian, Lorentzian, average between Gaussian and Lorentzian, Voigt fit, in one function with plots
###########and saving them as named figures including source name and date observed


workdir = pathlib.Path(r"C:\Users\debee\Downloads\git\IR_Variability_Of_BeXRBs_PhD\Scripts\GX304-1")
# Image_Dir= str(workdir)+"\\"+Source_Name
Image_Dir= str(workdir)
print(Image_Dir)

# .glob() produces a generator too
workdir.glob("*")


spectra_files=list(workdir.glob("smb*.txt"))
file_Names=[]
# print ((spectra_files))
for i in range(0,len(spectra_files)):
    file_Names.append(str(spectra_files[i])[66:])
print("File names:",file_Names)


guess = Smooth_find_double_peak_coords(x,y)

print(file)


for i in range(0,len(file_Names)):
    file = np.loadtxt(str(file_Names[i])) # smbxgpP201206270016_cr_cg_wr_01.txt smbxgpP201401200077_cr_cg_wr_01.txt smbxgpP202001120152_cr_cg_wr_01.txt
    Source_Name="GX304-1"
    x = file[:,0]
    y = file[:,1]
    guess = Smooth_find_double_peak_coords(x,y)

    nameString = file_Names[i][15:27] + '_' + Source_Name
    print(nameString)
    if(len(guess)==3):
        fit_ALL_t1(Source_Name, nameString, guess, x, y)
    if(len(guess)==6):
        fit_ALL_t2(Source_Name, nameString, guess, x, y)
    time.sleep(0.1)
    # break
print("end of plotting")

In [ ]:
########### SCIPY Gaussian fitting and plot:
# guess = [6555.0, 1.4, 6.0, 6561.0, -1.0, 3.0, 6565.0, 1.6, 4.0]
guess = Smooth_find_double_peak_coords(x,y)

popt, pcov = curve_fit(func_gaus, x, y, p0=guess,maxfev=20000)
print(popt)
fit_GAUS = func_gaus(x, *popt)

########## Error analysis and inclusion in model (can be excluded to go straight to plot):
########## Need to find the correct way to get SALT errors on the y data.
# yerr_data= np.sqrt((spec.flux/y_cont_fitted).value)
# fig=plt.figure(figsize=(16, 9)) #create the figure
# plt.xlabel('Wavelength (Angstroms)')
# plt.ylabel('Normalised flux')
# plt.xlim(6520,6600)
# plt.errorbar(spec.spectral_axis.value, (spec.flux/y_cont_fitted).value, yerr_data, ls='', color='k')
# plt.scatter(spec.spectral_axis.value, (spec.flux/y_cont_fitted).value, s=7, zorder=1000)

fig=plt.figure(figsize=(graph_W,graph_H)) #create the figure

plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Normalised flux')
plt.xlim(xmin,xmax)
plt.grid('minor')

plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
plt.plot(x, y, 'r-', linewidth=1,label='Normalised Spectra')
plt.plot(x, fit_GAUS, 'b-', linewidth=3, alpha=0.5,label='Gaussian Fit')
c=1
for i in range(0, len(popt), 3):
    y_temp = np.zeros_like(x)
    ctr = popt[i]
    amp = popt[i+1]
    wid = popt[i+2]
    y_temp = func_gaus(x, ctr, amp, wid)
    plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.5,label=f'individual Component {c}')
    c=c+1

plt.legend(fontsize=Legend_font_size)
plt.savefig('Gaussian_demo.png')
###########



In [ ]:
########### SCIPY Lorentzian fitting and plot:
# guess = [6555, 1.4, 6.0, 6561.0, -1.1, 3.0, 6565.0, 1.6, 4.0]
print(guess)

popt, pcov = curve_fit(func_lorentzian, x, y, p0=guess,maxfev=20000)
print(popt)
fit_LORERTZIAN = func_lorentzian(x, *popt)

plt.figure(figsize=(graph_W,graph_H)) #create the figure
plt.grid('minor')
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Normalised flux')
plt.xlim(xmin,xmax)
plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
plt.plot(x, fit_LORERTZIAN , 'b-', linewidth=3, alpha=0.5, label='Lorentzian Fit')
c=1
for i in range(0, len(popt), 3):
    y_temp = np.zeros_like(x)
    ctr = popt[i]
    amp = popt[i+1]
    wid = popt[i+2]
    y_temp = func_lorentzian(x, ctr, amp, wid)
    plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.5,label=f'individual Component {c}')
    c=c+1
plt.legend(fontsize=Legend_font_size)
plt.savefig('Lorentzian_demo.png')
###########

In [ ]:
########### SCIPY Average between GAUS and LORERNTZ fits, plot only:

fit_AVG=(fit_GAUS+fit_LORERTZIAN)/2

fig=plt.figure(figsize=(graph_W,graph_H)) #create the figure
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Normalised flux')
plt.xlim(xmin,xmax)
plt.grid('minor')
plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
plt.plot(x, fit_AVG , 'b-', linewidth=3, alpha=0.5,label='average between Gaussian and Lorentzian Fit')
plt.legend(fontsize=Legend_font_size)
plt.savefig('Average_GAUS_LORENTZ_demo.png')
###########

In [ ]:
########### SCIPY Voigt fitting and plot:
print(guess)

popt, pcov = curve_fit(func_voigt, x, y, p0=guess,maxfev=20000)
print(popt)
fit_VOIGT = func_voigt(x, *popt)

fig=plt.figure(figsize=(graph_W,graph_H)) #create the figure
plt.grid('minor')
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Normalised flux')
plt.xlim(xmin,xmax)
plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
plt.plot(x, fit_VOIGT , 'b-', linewidth=3, alpha=0.5,label='Voigt Fit')
c=1
for i in range(0, len(popt), 3):
    y_temp = np.zeros_like(x)
    ctr = popt[i]
    amp = popt[i+1]
    wid = popt[i+2]
    y_temp = func_voigt(x, ctr, amp, wid)
    plt.plot(x, y_temp, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.5,label=f'individual Component {c}')
    c=c+1
plt.legend(fontsize=Legend_font_size)
plt.savefig('Voigt_demo.png')
###########

In [ ]:
# Areas python script:
from scipy.integrate import quad
import numpy as np
import pylab as pl
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
#This script calculates the blue and red surface areas and these are compared with the measured equivalent widths
#Equations A.17 and A.18

file = np.loadtxt('HR_ave_velocities.dat')
file2 = np.loadtxt('HR_ave_EW.dat')


MJD = file[:,0]
#cycles = file[:,1]
v_blue = file[:,2]
v_red = file[:,3]

EW_blue = file2[:,2]
EW_red = file2[:,3]

MJD_eph = 43366.275


def integrand(x,eccen):
    return 1.0/((1.0 + eccen*np.cos(x))**2)

ep = 0.4

#print I[0]

a_in = 1.0/(1.0 - ep)
inc = 0.52
v_crit = 525.0
degtorad = np.pi/180.0
radtodeg = 180.0/np.pi

g = open("HR_Areas_ave_out.txt","w")
for i in range(len(MJD)):
    t1 = ((2.0*v_crit)/(v_red[i] - v_blue[i]))**2
    t2 = ((np.sin(inc))**2)/(1.0 - ep**2)
    a_p = t1*t2
    ts1 = 0.5*(a_p**2 - a_in**2)*(1.0 - ep**2)*np.cos(inc)
    MJD_conv = MJD[i] - 2400000.5
    ratio = ((v_red[i]+v_blue[i])/(v_red[i]-v_blue[i]))
    om = np.arccos(ratio*(1.0/ep))
    cos_om = ratio*(1.0/ep)
    f01 = np.arccos(-ep*cos_om) - om
    f01_deg = f01*radtodeg
    
    f02 = (2.0*np.pi - np.arccos(-ep*cos_om)) - om
    f02_deg = f02*radtodeg
    
    
    I_b = quad(integrand,f01,f02,args=(ep))
    ts2 = I_b[0]
    S_blue = ts1*ts2
    I_r = quad(integrand,f02,f01+2.0*np.pi,args=(ep))
    ts3 = I_r[0]
    S_red = ts1*ts3
    ratio_areas = S_blue/S_red
    ratio_EW = EW_blue[i]/EW_red[i]
    g.write("%0.3f  %0.3f   %0.3f   %0.3f   %0.3f   %0.3f   %0.3f   %0.3f   %0.3f   %0.3f   %0.3f\n" %(MJD[i],S_blue,S_red,EW_blue[i],EW_red[i],ratio_areas,ratio_EW,I_b[0],I_r[0],f01_deg,f02_deg))
g.close()




In [ ]:
#might be useful later to convert degrees to RA coordinates.
def map_DEG_to_RA(deg_list):
    RA_list=[]
    coord=0
    for i in range(0,len(deg_list)):
        hour= (deg_list[i]//1)*(1/15) #get the integer part of the float degree value and convert to hours. (1/15 hours per degree)
        minute= (hour-hour//1)*4.0 #decimal part of the degree float converted to minutes (4 minutes per degree)
        second= (minute-minute//1)*60 #decimal part of the minute coordinate, converted to seconds coordinate (60 sec per minute)
        coord=hour//1+(minute//1)/100+((second//1)//1)/10000
        RA_list.append(coord)
    return(RA_list)

print(map_DEG_to_RA([22.2565,23]))

In [ ]:
#Smoothing code
x = np.linspace(0,2*np.pi,100)
y = np.sin(x) + np.random.random(100) * 0.8

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

plt.plot(x, y,'o')
# plt.show()
# plt.close()
plt.plot(x, smooth(y,3), 'r-', lw=2)
# plt.show()
# plt.close()
plt.plot(x, smooth(y,19), 'g-', lw=2)
plt.show()
plt.close()

In [1]:
##### Main LMFit segment starts here with all imports and some settings

import pandas as pd
import pathlib
import random
import dateutil.parser
import time
import math
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=300 # highres display
from lmfit import Model
from lmfit.models import GaussianModel
from lmfit.models import LorentzianModel
from lmfit.models import VoigtModel
# from scipy.optimize import curve_fit
from scipy.special import voigt_profile
from scipy.signal import find_peaks

xmin,xmax = 6520,6600
ymin,ymax=-4.5,2
graph_W,graph_H = 8,5


In [ ]:
#####This segment opens the csv database and populates the spectrum filepaths and Julian dates, OVERWRITES THINGS, BE CAREFUL!
workdir = pathlib.Path.cwd() #get the initial working directory where this notebook is located
print("Initial work directory: ",workdir)
Source_name="GX304-1" #This is the source I am working with now
Image_Dir = workdir / (Source_name) #Add the source folder sub directory to save the work to
# print(f"Image saving directory for source {Source_name} is {Image_Dir}")

Source_database = pd.read_csv(f'{Image_Dir}\{Source_name}_dataframe.csv') #read the specific source database csv file
# print((f'{Image_Dir}\{Source_name}_dataframe.csv'))
# print(Source_database)

#populate the Spec_Path column with the spectra that are present in the Image_Dir directory:

spectra_files=list(Image_Dir.glob("smb*.txt"))
file_Names=[]
for i in range(0,len(spectra_files)):
    Source_database.loc[i,"Spec_Path"]=str(spectra_files[i])[74:] #Save the spectrum txt file path into the Spec_Path column
    dt = dateutil.parser.parse(str(spectra_files[i])[81:89]) #Parse the date from the file name string
    time = astropy.time.Time(dt) 
    Source_database.loc[i,"JD"]=str(int(time.jd)) #Save into the database as the Julian date for timeseries plots later
    file_Names.append(str(spectra_files[i])[74:])
print("File names:",Source_database["JD"])

Source_database.to_csv(f'{Image_Dir}\{Source_name}_dataframe.csv',index=False) #THIS SAVES AND OVERWRITES WHATEVER VALUES HAVE BEEN READ INTO THE DATABASE! BE CAREFUL WITH IT.



In [ ]:
##### REDUNDANT, MIGHT DELETE THIS CELL!  This segmnent reads from the csv database and analyses the spectra

workdir = pathlib.Path.cwd() #get the initial working directory where this notebook is located
print("Initial work directory: ",workdir)
Source_name="GX304-1" #This is the source I am working with now
Image_Dir = workdir / (Source_name) #Add the source folder sub directory to save the work to
# print(f"Image saving directory for source {Source_name} is {Image_Dir}")

Source_database = pd.read_csv(f'{Image_Dir}\{Source_name}_dataframe.csv') #read the specific source database csv file

for i in range(0,len(Source_database["JD"])):

    p=Source_database["Spec_Path"][i]
    print(f"{Image_Dir}\{p}")
    file = np.loadtxt(str(f"{Image_Dir}\{p}")) # This imports the wavelength and normalised flux lists from each spectrum file listed in the database
    x = file[:,0]
    y = file[:,1]

    #If statements to determine which kind of fit to do depending on "type" or the shape of the Halpha line profile
    if(Source_database["Type"][i]==1):
        print("Single emission peak type spectrum")

    if(Source_database["Type"][i]==2):
        print("Double emission peak type spectrum")

    if(Source_database["Type"][i]==3):
        print("Single absorption valley type spectrum")

    if(Source_database["Type"][i]==4):
        print("Absorption valley walled by emission wings type spectrum")

    if(Source_database["Type"][i]==5):
        print("Emission peak walled by absorption wings type spectrum")

    # guess = Smooth_find_double_peak_coords(x,y)

    # nameString = file_Names[i][15:27] + '_' + Source_Name
    # print(nameString)
    # if(len(guess)==3):
    #     fit_ALL_t1(Source_Name, nameString, guess, x, y)
    # if(len(guess)==6):
    #     fit_ALL_t2(Source_Name, nameString, guess, x, y)
    time.sleep(0.1)
    # break
print("end of plotting")

In [27]:
#ALL THE FUNCTION DEFS

printTypeOnGraph=1 #This variable tells the plots to indicate the type of spectrum from 1 to 5 if variable is set to 1.
printFWHMOnGraph=0 #This variable tells the plots to show the FWHM if variable is set to 1.
printHalphaOnGraph=1 #This variable tells the plots to show rest wavelength of H alpha if variable is set to 1.
printVtoRratioOnGraph=1 #This variable tells the plots to show the peaks used in V/R calculations and peak seperation if variable is set to 1.
printModelUncertaintyOnGraph=1 #This variable tells the plots to show the uncertainty in the model values fitted to the data if variable is set to 1. Set sigma level below
ModelUncertaintySigmaLevel = 3 #sigma error level to indicate if above variable is set to 1
Legend_font_size=7



def LMFIT_gaussian(x, amp, cen, wid): # as defined in documentation https://lmfit.github.io/lmfit-py/builtin_models.html#gaussianmodel

    return (amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))

def LMFIT_lorentzian(x, amp, cen, wid): # as defined in documentation https://lmfit.github.io/lmfit-py/builtin_models.html#lorentzianmodel

    return ((amp/np.pi)*(wid/((x-cen)**2+wid**2)))

#The Voigt model is internally defined for LMfit modules, see https://lmfit.github.io/lmfit-py/builtin_models.html#voigtmodel
def func_voigt(x, amp, ctr, wid): #Fitting function voigt

    y = np.zeros_like(x)

    gam = wid
    y = y + voigt_profile(x - ctr, wid, gam) * amp
    return y

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


def Smooth_find_single_valley_coords(x_data,y_data): #Function to find the wavelength coordinate of a single valley
    print("We are in the valley finder!")

    halpha_region_x = x_data[(x_data >= 6553) & (x_data <= 6575)]
    halpha_region_y = y_data[(x_data >= 6553) & (x_data <= 6575)]

    halpha_region_y=smooth(halpha_region_y,6)

    dip_idx = np.argmin(halpha_region_y[(halpha_region_x >= 6553) & (halpha_region_x <= 6572)])
    dip_wavelength = halpha_region_x[(halpha_region_x >= 6553) & (halpha_region_x <= 6572)][dip_idx]

    return dip_wavelength

def Smooth_find_single_peak_coords(x_data,y_data): #Function to find the wavelength coordinate of a single peak
    halpha_region_x = x_data[(x_data >= 6553) & (x_data <= 6575)]
    halpha_region_y = y_data[(x_data >= 6553) & (x_data <= 6575)]

    halpha_region_y=smooth(halpha_region_y,6)

    peak_idx = np.argmax(halpha_region_y[(halpha_region_x >= 6553) & (halpha_region_x <= 6572)])
    peak_wavelength = halpha_region_x[(halpha_region_x >= 6553) & (halpha_region_x <= 6572)][peak_idx]

    return peak_wavelength

def Smooth_find_double_peak_coords(x_data,y_data): #Function to find P_0 guess values for double or single peak plotting
    halpha_region_x = x_data[(x_data > 6553) & (x_data < 6575)]
    halpha_region_y = y_data[(x_data > 6553) & (x_data < 6575)]

    halpha_region_y=smooth(halpha_region_y,3)

    peaks, _ = find_peaks(halpha_region_y,distance=2)
    print(peaks)
    peak_wavelengths=halpha_region_x[peaks]
    peak_fluxes=halpha_region_y[peaks]

    if len(peaks)>1:
        # Sort peaks by flux
        sorted_peaks = np.argsort(peak_fluxes)[-2:]  # take top 2
        selected_peaks = [(peak_wavelengths[i], peak_fluxes[i]) for i in sorted_peaks]

        # For the valley, just take min flux between the two peaks
        left, right = np.min([p[0] for p in selected_peaks]), np.max([p[0] for p in selected_peaks])
        mask = (halpha_region_x >= left) & (halpha_region_x <= right)
        dip_idx = np.argmin(halpha_region_y[mask])
        dip_wavelength = halpha_region_x[mask][dip_idx] #This is basically the location of the centroid?
        dip_flux = halpha_region_y[mask][dip_idx]

        # print("Peaks (λ, flux):", selected_peaks)
        # print("Valley (λ, flux):", (dip_wavelength, dip_flux))

        ctr1=math.trunc(selected_peaks[0][0]*10)
        amp1=math.trunc(selected_peaks[0][1]*10)
        wid1=6*10
        ctr2=dip_wavelength
        ctr2=math.trunc(ctr2*10)
        wid2=0.3*((dip_wavelength-peak_wavelengths[0])+(peak_wavelengths[1]-dip_wavelength))
        wid2=math.trunc(3*10)
        ctr3=math.trunc(selected_peaks[1][0]*10)
        amp3=math.trunc(selected_peaks[1][1]*10)
        wid3=4*10
        amp2=dip_flux
        # amp2=-((amp1/10+amp3/10)/2-amp2)*5
        amp2=math.trunc(amp2*0.76689539232*10)

        # result=[ctr1/10,amp1/10,wid1/10,ctr2/10,amp2/10,wid2/10,ctr3/10,amp3/10,wid3/10]
        result=[ctr1/10,amp1/10-1,wid1/10,ctr3/10,amp3/10-1,wid3/10]
        print("2 component guess: ",result[0],result[3])

    if (len(peaks)>2 | len(peaks)<1):
        print("Something went wrong in peak finiding")
        result="something went wrong"
        

    return [result[0],result[3]] #returns blue peak central wavelength, then red peak central wavelength


def LMFIT_t1_G(Source_Name,nameString,x,y): # Fit Gaussian function for the type 1 spectra with single emission peak

    gmodel1 = GaussianModel(prefix='g1_') # Define the single peak gaussian component

    model = gmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_peak_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['g1_amplitude'].set(value=8, min=0, max=12)
    params['g1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['g1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['g1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Gaussian Fit')
    if (printTypeOnGraph==1):
        plt.title("T1")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()

    return [result.params['g1_height'].value, result.params['g1_center'].value, result.params['g1_sigma'].value]

def LMFIT_t1_L(Source_Name,nameString,x,y): # Fit Lorentzian function for the type 1 spectra with single emission peak

    lmodel1 = LorentzianModel(prefix='l1_') # Define the single peak lorentzian component

    model = lmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_peak_coords(x,y)

    # Set initial guesses for the first Lorentzian
    params['l1_amplitude'].set(value=8, min=0, max=12)
    params['l1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['l1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['l1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Lorentzian Fit')
    if (printTypeOnGraph==1):
        plt.title("T1")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()

    return [result.params['l1_height'].value, result.params['l1_center'].value, result.params['l1_sigma'].value]

def LMFIT_t1_V(Source_Name,nameString,x,y): # Fit Voigt function for the type 1 spectra with single emission peak

    vmodel1 = VoigtModel(prefix='v1_') # Define the single peak voigt component

    model = vmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_peak_coords(x,y)

    # Set initial guesses for the first Voigt
    params['v1_amplitude'].set(value=8, min=0, max=12)
    params['v1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['v1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['v1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Voigt Fit')
    if (printTypeOnGraph==1):
        plt.title("T1")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()

    return [result.params['v1_height'].value, result.params['v1_center'].value, result.params['v1_sigma'].value]


def LMFIT_t2_G(Source_Name,nameString,x,y): # Fit Gaussian function for the type 2 spectra with double emission peak

    gmodel1 = GaussianModel(prefix='g1_') # Define the first peak gaussian component
    gmodel2 = GaussianModel(prefix='g2_') # Define the second peak gaussian component

    model = gmodel1 + gmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_double_peak_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['g1_amplitude'].set(value=8, min=1, max=12)
    params['g1_center'].set(value=centroid_guess[0], min=centroid_guess[0]-2, max=centroid_guess[0]+2)
    params['g1_sigma'].set(value=3, min=0, max=7)

    # Set initial guesses for the second Gaussian
    params['g2_amplitude'].set(value=8, min=1, max=12)
    params['g2_center'].set(value=centroid_guess[1], min=centroid_guess[1]-2, max=centroid_guess[1]+2)
    params['g2_sigma'].set(value=3, min=0, max=7)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['g1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_gaussian(x, result.params['g1_amplitude'].value, result.params['g1_center'].value, result.params['g1_sigma'].value)
    result2= LMFIT_gaussian(x, result.params['g2_amplitude'].value, result.params['g2_center'].value, result.params['g2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Gaussian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T2")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printVtoRratioOnGraph==1):
        x_temp1= result.params['g1_center'].value
        y_temp1= max(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        idx = np.argmax(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        x_temp1 = x[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)][idx]
        plt.axvline(x = x_temp1, color = 'black', linewidth=0.5)
        plt.scatter(x_temp1, y_temp1, color = 'black', linewidth=2)
        x_temp2= result.params['g2_center'].value
        y_temp2= max(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        idx = np.argmax(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        x_temp2 = x[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)][idx]
        plt.axvline(x = x_temp2, color = 'black', linewidth=0.5)
        plt.scatter(x_temp2, y_temp2, color = 'black', linewidth=2, label=f"V/R {np.round(y_temp1/y_temp2,3)} Peaksep {np.round(x_temp2-x_temp1,3)}")
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()

    return [result.params['g1_height'].value, result.params['g1_center'].value, result.params['g1_sigma'].value,
            result.params['g2_height'].value, result.params['g2_center'].value, result.params['g2_sigma'].value]

def LMFIT_t2_L(Source_Name,nameString,x,y): # Fit Lorentzian function for the type 2 spectra with double emission peak

    lmodel1 = LorentzianModel(prefix='l1_') # Define the first peak Lorentzian component
    lmodel2 = LorentzianModel(prefix='l2_') # Define the second peak Lorentzian component

    model = lmodel1 + lmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_double_peak_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['l1_amplitude'].set(value=8, min=1, max=12)
    params['l1_center'].set(value=centroid_guess[0], min=centroid_guess[0]-2, max=centroid_guess[0]+2)
    params['l1_sigma'].set(value=3, min=0, max=7)

    # Set initial guesses for the second Gaussian
    params['l2_amplitude'].set(value=8, min=1, max=12)
    params['l2_center'].set(value=centroid_guess[1], min=centroid_guess[1]-2, max=centroid_guess[1]+2)
    params['l2_sigma'].set(value=3, min=0, max=7)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_lorentzian(x, result.params['l1_amplitude'].value, result.params['l1_center'].value, result.params['l1_sigma'].value)
    result2= LMFIT_lorentzian(x, result.params['l2_amplitude'].value, result.params['l2_center'].value, result.params['l2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Lorentzian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T2")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printVtoRratioOnGraph==1):
        x_temp1= result.params['l1_center'].value
        y_temp1= max(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        idx = np.argmax(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        x_temp1 = x[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)][idx]
        plt.axvline(x = x_temp1, color = 'black', linewidth=0.5)
        plt.scatter(x_temp1, y_temp1, color = 'black', linewidth=2)
        x_temp2= result.params['l2_center'].value
        y_temp2= max(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        idx = np.argmax(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        x_temp2 = x[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)][idx]
        plt.axvline(x = x_temp2, color = 'black', linewidth=0.5)
        plt.scatter(x_temp2, y_temp2, color = 'black', linewidth=2, label=f"V/R {np.round(y_temp1/y_temp2,3)} Peaksep {np.round(x_temp2-x_temp1,3)}")
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()

    return [result.params['l1_height'].value, result.params['l1_center'].value, result.params['l1_sigma'].value,
            result.params['l2_height'].value, result.params['l2_center'].value, result.params['l2_sigma'].value]

def LMFIT_t2_V(Source_Name,nameString,x,y): # Fit Voigt function for the type 2 spectra with double emission peak

    vmodel1 = VoigtModel(prefix='v1_') # Define the first peak Voigt component
    vmodel2 = VoigtModel(prefix='v2_') # Define the second peak Voigt component

    model = vmodel1 + vmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_double_peak_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['v1_amplitude'].set(value=8, min=1, max=12)
    params['v1_center'].set(value=centroid_guess[0], min=centroid_guess[0]-2, max=centroid_guess[0]+2)
    params['v1_sigma'].set(value=3, min=0, max=7)

    # Set initial guesses for the second Gaussian
    params['v2_amplitude'].set(value=8, min=1, max=12)
    params['v2_center'].set(value=centroid_guess[1], min=centroid_guess[1]-2, max=centroid_guess[1]+2)
    params['v2_sigma'].set(value=3, min=0, max=7)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= func_voigt(x, result.params['v1_amplitude'].value, result.params['v1_center'].value, result.params['v1_sigma'].value)
    result2= func_voigt(x, result.params['v2_amplitude'].value, result.params['v2_center'].value, result.params['v2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Voigt best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T2")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printVtoRratioOnGraph==1):
        x_temp1= result.params['v1_center'].value
        y_temp1= max(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        idx = np.argmax(y[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)])
        x_temp1 = x[(x >= x_temp1-1.5) & (x <= x_temp1+1.5)][idx]
        plt.axvline(x = x_temp1, color = 'black', linewidth=0.5)
        plt.scatter(x_temp1, y_temp1, color = 'black', linewidth=2)
        x_temp2= result.params['v2_center'].value
        y_temp2= max(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        idx = np.argmax(y[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)])
        x_temp2 = x[(x >= x_temp2-1.5) & (x <= x_temp2+1.5)][idx]
        plt.axvline(x = x_temp2, color = 'black', linewidth=0.5)
        plt.scatter(x_temp2, y_temp2, color = 'black', linewidth=2, label=f"V/R {np.round(y_temp1/y_temp2,3)} Peaksep {np.round(x_temp2-x_temp1,3)}")
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()

    return [result.params['v1_height'].value, result.params['v1_center'].value, result.params['v1_sigma'].value,
            result.params['v2_height'].value, result.params['v2_center'].value, result.params['v2_sigma'].value]


def LMFIT_t3_G(Source_Name,nameString,x,y): # Fit Gaussian function for the type 3 spectra with single absorption valley

    gmodel1 = GaussianModel(prefix='g1_') # Define the single valley gaussian component

    model = gmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_valley_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['g1_amplitude'].set(value=-2, max=-0.5, min=-35)
    params['g1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['g1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['g1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Gaussian Fit')
    if (printTypeOnGraph==1):
        plt.title("T3")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()

    return [result.params['g1_height'].value, result.params['g1_center'].value, result.params['g1_sigma'].value]

def LMFIT_t3_L(Source_Name,nameString,x,y): # Fit Lorentzian function for the type 3 spectra with single absorption valley

    lmodel1 = LorentzianModel(prefix='l1_') # Define the single valley lorentzian component

    model = lmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_valley_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['l1_amplitude'].set(value=-2, max=-0.5, min=-35)
    params['l1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['l1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['l1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Lorentzian Fit')
    if (printTypeOnGraph==1):
        plt.title("T3")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()

    return [result.params['l1_height'].value, result.params['l1_center'].value, result.params['l1_sigma'].value]

def LMFIT_t3_V(Source_Name,nameString,x,y): # Fit Voigt function for the type 3 spectra with single absorption valley

    vmodel1 = VoigtModel(prefix='v1_') # Define the single valley voigt component

    model = vmodel1 # This is redundant, but only done for consistancy with multi component fits

    params = model.make_params() # Set up initial parameters and perform the fit

    centroid_guess=Smooth_find_single_valley_coords(x,y)

    # Set initial guesses for the first Gaussian
    params['v1_amplitude'].set(value=-2, max=-0.5, min=-35)
    params['v1_center'].set(value=centroid_guess, min=centroid_guess-2, max=centroid_guess+2)
    params['v1_sigma'].set(value=10, min=0)
    
    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit

    print(result.params['v1_height'].value) 
    print(result.fit_report()) # Print results report to see how good the fitting went

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Voigt Fit')
    if (printTypeOnGraph==1):
        plt.title("T3")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()

    return [result.params['v1_height'].value, result.params['v1_center'].value, result.params['v1_sigma'].value]


def LMFIT_t4_G(Source_Name,nameString,x,y): # Fit Gaussian function for the type 4 spectra with an absorption valley, winged by emission

    gmodel1 = GaussianModel(prefix='g1_') # Define the first peak gaussian component
    gmodel2 = GaussianModel(prefix='g2_') # Define the second peak gaussian component

    model = gmodel1 + gmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Gaussian
    params['g1_amplitude'].set(value=-40, max=0, min=-200)
    params['g1_center'].set(value=6563, min=6558, max=6566)
    params['g1_sigma'].set(value=6, min=0, max=8)

    # Set initial guesses for the second Gaussian
    params['g2_amplitude'].set(value=20, min=0, max = 50)
    params['g2_center'].set(value=6563, min=6550, max=6570)
    params['g2_sigma'].set(value=9, min=0, max=15)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['g1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_gaussian(x, result.params['g1_amplitude'].value, result.params['g1_center'].value, result.params['g1_sigma'].value)
    result2= LMFIT_gaussian(x, result.params['g2_amplitude'].value, result.params['g2_center'].value, result.params['g2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    # plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Gaussian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T4")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()

    return [result.params['g1_height'].value, result.params['g1_center'].value, result.params['g1_sigma'].value,
            result.params['g2_height'].value, result.params['g2_center'].value, result.params['g2_sigma'].value]

def LMFIT_t4_L(Source_Name,nameString,x,y): # Fit Lorentzian function for the type 4 spectra with an absorption valley, winged by emission

    lmodel1 = LorentzianModel(prefix='l1_') # Define the first peak Lorentzian component
    lmodel2 = LorentzianModel(prefix='l2_') # Define the second peak Lorentzian component

    model = lmodel1 + lmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Lorentzian
    params['l1_amplitude'].set(value=-40, max=0, min=-200)
    params['l1_center'].set(value=6563, min=6558, max=6566)
    params['l1_sigma'].set(value=6, min=0, max=8)

    # Set initial guesses for the second Gaussian
    params['l2_amplitude'].set(value=20, min=0, max = 50)
    params['l2_center'].set(value=6563, min=6550, max=6570)
    params['l2_sigma'].set(value=9, min=0, max=15)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_lorentzian(x, result.params['l1_amplitude'].value, result.params['l1_center'].value, result.params['l1_sigma'].value)
    result2= LMFIT_lorentzian(x, result.params['l2_amplitude'].value, result.params['l2_center'].value, result.params['l2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Lorentzian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T4")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()

    return [result.params['l1_height'].value, result.params['l1_center'].value, result.params['l1_sigma'].value,
            result.params['l2_height'].value, result.params['l2_center'].value, result.params['l2_sigma'].value]

def LMFIT_t4_V(Source_Name,nameString,x,y): # Fit Voigt function for the type 4 spectra with an absorption valley, winged by emission

    vmodel1 = VoigtModel(prefix='v1_') # Define the first peak Voigt component
    vmodel2 = VoigtModel(prefix='v2_') # Define the second peak Voigt component

    model = vmodel1 + vmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Voigt
    params['v1_amplitude'].set(value=-40, max=0, min=-200)
    params['v1_center'].set(value=6563, min=6558, max=6566)
    params['v1_sigma'].set(value=6, min=0, max=8)

    # Set initial guesses for the second Gaussian
    params['v2_amplitude'].set(value=20, min=0, max = 50)
    params['v2_center'].set(value=6563, min=6550, max=6570)
    params['v2_sigma'].set(value=9, min=0, max=15)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= func_voigt(x, result.params['v1_amplitude'].value, result.params['v1_center'].value, result.params['v1_sigma'].value)
    result2= func_voigt(x, result.params['v2_amplitude'].value, result.params['v2_center'].value, result.params['v2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Voigt best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T4")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()

    return [result.params['v1_height'].value, result.params['v1_center'].value, result.params['v1_sigma'].value,
            result.params['v2_height'].value, result.params['v2_center'].value, result.params['v2_sigma'].value]


def LMFIT_t5_G(Source_Name,nameString,x,y): # Fit Gaussian function for the type 5 spectra with an emission peak, winged by absorption

    gmodel1 = GaussianModel(prefix='g1_') # Define the first peak gaussian component
    gmodel2 = GaussianModel(prefix='g2_') # Define the second peak gaussian component

    model = gmodel1 + gmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Gaussian
    params['g1_amplitude'].set(value=-40, max=0, min=-200)
    params['g1_center'].set(value=6563, min=6558, max=6570)
    params['g1_sigma'].set(value=9, min=8, max=15)

    # Set initial guesses for the second Gaussian
    params['g2_amplitude'].set(value=20, min=0, max = 50)
    params['g2_center'].set(value=6563, min=6555, max=6570)
    params['g2_sigma'].set(value=3, min=0, max=5)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['g1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_gaussian(x, result.params['g1_amplitude'].value, result.params['g1_center'].value, result.params['g1_sigma'].value)
    result2= LMFIT_gaussian(x, result.params['g2_amplitude'].value, result.params['g2_center'].value, result.params['g2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    # plt.ylim(min(y[(x > 6553) & (x < 6572)])-0.2,max((y[(x > 6550) & (x < 6575)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Gaussian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T5")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_GaussianFit.png')
    plt.close()

    return [result.params['g1_height'].value, result.params['g1_center'].value, result.params['g1_sigma'].value,
            result.params['g2_height'].value, result.params['g2_center'].value, result.params['g2_sigma'].value]

def LMFIT_t5_L(Source_Name,nameString,x,y): # Fit Lorentzian function for the type 5 spectra with an emission peak, winged by absorption

    lmodel1 = LorentzianModel(prefix='l1_') # Define the first peak Lorentzian component
    lmodel2 = LorentzianModel(prefix='l2_') # Define the second peak Lorentzian component

    model = lmodel1 + lmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Lorentzian
    params['l1_amplitude'].set(value=-40, max=0, min=-200)
    params['l1_center'].set(value=6563, min=6558, max=6570)
    params['l1_sigma'].set(value=9, min=8, max=15)

    # Set initial guesses for the second Gaussian
    params['l2_amplitude'].set(value=20, min=0, max = 50)
    params['l2_center'].set(value=6563, min=6555, max=6570)
    params['l2_sigma'].set(value=3, min=0, max=5)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= LMFIT_lorentzian(x, result.params['l1_amplitude'].value, result.params['l1_center'].value, result.params['l1_sigma'].value)
    result2= LMFIT_lorentzian(x, result.params['l2_amplitude'].value, result.params['l2_center'].value, result.params['l2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Lorentzian best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T5")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_LorentzianFit.png')
    plt.close()

    return [result.params['l1_height'].value, result.params['l1_center'].value, result.params['l1_sigma'].value,
            result.params['l2_height'].value, result.params['l2_center'].value, result.params['l2_sigma'].value]

def LMFIT_t5_V(Source_Name,nameString,x,y): # Fit Voigt function for the type 5 spectra with an emission peak, winged by absorption

    vmodel1 = VoigtModel(prefix='v1_') # Define the first peak Voigt component
    vmodel2 = VoigtModel(prefix='v2_') # Define the second peak Voigt component

    model = vmodel1 + vmodel2  # Combine them into a composite model

    params = model.make_params() # Set up initial parameters and perform the fit

    # Set initial guesses for the first Voigt
    params['v1_amplitude'].set(value=-40, max=0, min=-200)
    params['v1_center'].set(value=6563, min=6558, max=6570)
    params['v1_sigma'].set(value=9, min=8, max=15)

    # Set initial guesses for the second Gaussian
    params['v2_amplitude'].set(value=20, min=0, max = 50)
    params['v2_center'].set(value=6563, min=6555, max=6570)
    params['v2_sigma'].set(value=3, min=0, max=5)

    result = model.fit(y, params, x=x, max_nfev=10000) # Perform the fit
    # print(result.params['l1_amplitude'].value)

    print(result.fit_report()) # Print results report to see how good the fitting went

    result1= func_voigt(x, result.params['v1_amplitude'].value, result.params['v1_center'].value, result.params['v1_sigma'].value)
    result2= func_voigt(x, result.params['v2_amplitude'].value, result.params['v2_center'].value, result.params['v2_sigma'].value)

    plt.figure(figsize=(graph_W,graph_H)) # create the figure
    plt.xlabel('Wavelength (Angstroms)')
    plt.ylabel('Normalised flux')
    plt.xlim(xmin,xmax)
    plt.ylim(min(y[(x > xmin) & (x < xmax)])-0.2,max((y[(x > xmin) & (x < xmax)]))+0.2)
    plt.grid('minor')
    plt.plot(x, y, 'r--', linewidth=1, label="SALT data")
    plt.plot(x, result.best_fit , 'b-', linewidth=2, alpha=0.5,label='Voigt best fit')
    plt.plot(x, result1, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 1')
    plt.plot(x, result2, color=(random.random(), random.random(), random.random()), linewidth=3, alpha=0.3,label='individual Component 2')
    if (printTypeOnGraph==1):
        plt.title("T5")
    if (printHalphaOnGraph==1):
        plt.axvline(x = 6562.8, color = 'orange', linewidth=3, alpha=0.2, label='Hα')
    if (printModelUncertaintyOnGraph==1):
        dely = result.eval_uncertainty(sigma=ModelUncertaintySigmaLevel)
        plt.fill_between(x, result.best_fit-dely, result.best_fit+dely, color="#ABABAB",
                 label=r'3-$\sigma$ uncertainty band', alpha=0.5)
    plt.legend(fontsize=Legend_font_size)
    plt.savefig('.\\'+ Source_Name +'\\'+ nameString + '_VoigtFit.png')
    plt.close()

    return [result.params['v1_height'].value, result.params['v1_center'].value, result.params['v1_sigma'].value,
            result.params['v2_height'].value, result.params['v2_center'].value, result.params['v2_sigma'].value]


#These all do what their names say. The fit all 3 types of functions depending on the type specified in the database, and save the figures in the working directory
def LMfit_ALL_t1(Source_Name, nameString, x, y):
    ########### Gaussian fitting and plot:
    LMFIT_t1_G(Source_Name,nameString,x,y)
    ###########

    ########### Lorentzian fitting and plot:
    LMFIT_t1_L(Source_Name,nameString,x,y)
    ###########

    ########### Voigt fitting and plot:
    LMFIT_t1_V(Source_Name,nameString,x,y)
    ###########

    
    return ("finished "+nameString)

def LMfit_ALL_t2(Source_Name, nameString, x, y):
    ########### Gaussian fitting and plot:
    LMFIT_t2_G(Source_Name,nameString,x,y)
    ###########

    ########### Lorentzian fitting and plot:
    LMFIT_t2_L(Source_Name,nameString,x,y)
    ###########

    ########### Voigt fitting and plot:
    LMFIT_t2_V(Source_Name,nameString,x,y)
    ###########

def LMfit_ALL_t3(Source_Name, nameString, x, y):
    ########### Gaussian fitting and plot:
    LMFIT_t3_G(Source_Name,nameString,x,y)
    ###########

    ########### Lorentzian fitting and plot:
    LMFIT_t3_L(Source_Name,nameString,x,y)
    ###########

    ########### Voigt fitting and plot:
    LMFIT_t3_V(Source_Name,nameString,x,y)
    ###########

def LMfit_ALL_t4(Source_Name, nameString, x, y):
    ########### Gaussian fitting and plot:
    LMFIT_t4_G(Source_Name,nameString,x,y)
    ###########

    ########### Lorentzian fitting and plot:
    LMFIT_t4_L(Source_Name,nameString,x,y)
    ###########

    ########### Voigt fitting and plot:
    LMFIT_t4_V(Source_Name,nameString,x,y)
    ###########

def LMfit_ALL_t5(Source_Name, nameString, x, y):
    ########### Gaussian fitting and plot:
    LMFIT_t5_G(Source_Name,nameString,x,y)
    ###########

    ########### Lorentzian fitting and plot:
    LMFIT_t5_L(Source_Name,nameString,x,y)
    ###########

    ########### Voigt fitting and plot:
    LMFIT_t5_V(Source_Name,nameString,x,y)
    ###########
    
    return ("finished "+nameString)

In [28]:
##### This segmnent reads from the csv database and analyses the spectra

workdir = pathlib.Path.cwd() #get the initial working directory where this notebook is located
print("Initial work directory: ",workdir)
Source_Name="GX304-1" #This is the source I am working with now
Image_Dir = workdir / (Source_Name) #Add the source folder sub directory to save the work to
# print(f"Image saving directory for source {Source_Name} is {Image_Dir}")

Source_database = pd.read_csv(f'{Image_Dir}\{Source_Name}_dataframe.csv') #read the specific source database csv file

spectra_files=list(Image_Dir.glob("smb*.txt"))
file_Names=[]
# print ((spectra_files))
for i in range(0,len(spectra_files)):
    file_Names.append(str(spectra_files[i])[66:])
print("File names:",file_Names)

for i in range(0,len(Source_database["JD"])):

    p=Source_database["Spec_Path"][i]
    print(f"{Image_Dir}\{p}")
    file = np.loadtxt(str(f"{Image_Dir}\{p}")) # This imports the wavelength and normalised flux lists from each spectrum file listed in the database
    x = file[:,0]
    y = file[:,1]
    
    nameString = file_Names[i][15:27] + '_' + Source_Name

    #If statements to determine which kind of fit to do depending on "type" or the shape of the Halpha line profile
    if(Source_database["Type"][i]==1):
        print("Single emission peak type spectrum")
        # LMfit_ALL_t1(Source_Name,nameString,x,y)

    if(Source_database["Type"][i]==2):
        print("Double emission peak type spectrum")
        LMfit_ALL_t2(Source_Name, nameString, x, y)
        # break

    if(Source_database["Type"][i]==3):
        print("Single absorption valley type spectrum")
        # LMfit_ALL_t3(Source_Name, nameString, x, y)

    if(Source_database["Type"][i]==4):
        print("Absorption valley walled by emission wings type spectrum")
        # LMfit_ALL_t4(Source_Name, nameString, x, y)


    if(Source_database["Type"][i]==5):
        print("Emission peak walled by absorption wings type spectrum")
        # LMfit_ALL_t5(Source_Name, nameString, x, y)
        # break

    time.sleep(0.1)
    # if (Source_database["Type"][i]==5):
    #     break
    # break
print("end of plotting")

Initial work directory:  c:\Users\debee\Downloads\git\IR_Variability_Of_BeXRBs_PhD\Scripts
File names: ['GX304-1\\smbxgpP201206270016_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201206270017_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201301240111_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201301240112_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201303060077_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201303060078_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201303230069_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201303230070_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201304270055_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201304270056_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201401200076_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201401200077_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201402160001_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201402160002_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201402270028_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201402270029_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201403100009_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201403100010_cr_cg_wr_01.txt', 'GX304-1\\smbxgpP201403220152_cr

c:\Python\lib\site-packages\lmfit\model.py:1760: RuntimeWarning: invalid value encountered in sqrt
  self.dely = scale * np.sqrt(df2_total)
c:\Python\lib\site-packages\lmfit\model.py:1765: RuntimeWarning: invalid value encountered in sqrt
  self.dely_comps[key] = scale * np.sqrt(df2[key])


[23 41]
2 component guess:  6558.9 6563.5
[[Model]]
    (Model(lorentzian, prefix='l1_') + Model(lorentzian, prefix='l2_'))
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 78
    # data points      = 3170
    # variables        = 6
    chi-square         = 2.66358294
    reduced chi-square = 8.4184e-04
    Akaike info crit   = -22437.3525
    Bayesian info crit = -22400.9836
    R-squared          = 0.90554048
[[Variables]]
    l1_amplitude:  3.10983759 +/- 0.21842046 (7.02%) (init = 8)
    l1_center:     6558.15811 +/- 0.07677265 (0.00%) (init = 6558.9)
    l1_sigma:      2.25076394 +/- 0.13164105 (5.85%) (init = 3)
    l2_amplitude:  9.76615134 +/- 0.25396351 (2.60%) (init = 8)
    l2_center:     6563.83435 +/- 0.06214726 (0.00%) (init = 6563.5)
    l2_sigma:      3.54003137 +/- 0.07961792 (2.25%) (init = 3)
    l1_fwhm:       4.50152788 +/- 0.26328211 (5.85%) == '2.0000000*l1_sigma'
    l1_height:     0.43980272 +/- 0.01465168 (3.33%) == '0.3183099*l1_am